In [1]:
import os
import pandas as pd
import numpy as np

train_data = pd.read_csv("data/training_data_utf8.csv")

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
date = train_data.transaction_ym.apply(pd.to_datetime, format="%Y-%m")
relativeShare = train_data.plot_area * train_data.plot_share

In [3]:
train_data["relativeShare"] = relativeShare
train_data["transaction_month"] = date.dt.month
train_data["transaction_year"] = date.dt.year

In [4]:
train_data_clean = train_data[["price_per_m2", "transaction_year", "transaction_month", "relativeShare", "property_type", "cadastral", "contract_type", "land_use", "protection_zone", "building_ban"]]
train_data_clean = train_data_clean.dropna()
train_data_clean.shape

(44260, 10)

In [5]:
#describe numerical values
train_data_clean.describe()

,price_per_m2,transaction_year,transaction_month,relativeShare
count,44260.000000,44260.000000,44260.000000,4.426000e+04
mean,789.511534,1999.308586,6.914822,1.698198e+03
std,2278.670920,7.922187,3.417283,1.009561e+04
min,0.161937,1987.000000,1.000000,7.601351e-02
25%,181.682086,1992.000000,4.000000,3.840000e+02
50%,354.343664,1999.000000,7.000000,5.540000e+02
75%,726.728421,2006.000000,10.000000,9.760000e+02
max,197628.458498,2013.000000,12.000000,1.187781e+06


In [6]:
#describe categorical values
train_data_clean.describe(include="O")

,property_type,cadastral,contract_type,land_use,protection_zone,building_ban
count,44260,44260,44260,44260,44260,44260
unique,16,88,11,56,2,2
top,unbebaut,Eßling,Kaufvertrag,WI/GBI,No,No
freq,15319,2260,38754,12099,43200,43768


In [7]:
corrmat = train_data_clean.corr()
corrmat["price_per_m2"].sort_values(ascending=False)

price_per_m2         1.000000
transaction_year     0.132131
transaction_month    0.028067
relativeShare       -0.026065
Name: price_per_m2, dtype: float64

In [8]:
data = train_data_clean.drop("price_per_m2", axis=1)
labels = train_data_clean["price_per_m2"].copy()

In [9]:
train_data_cat = train_data_clean[["property_type", "cadastral", "contract_type", "land_use", "protection_zone", "building_ban"]]

In [10]:
train_data_num = train_data_clean[["transaction_year", "relativeShare"]]

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

In [12]:
from future_encoders import ColumnTransformer
from future_encoders import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

num_attribs = list(train_data_num)
cat_attribs = list(train_data_cat)

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs)
])

In [13]:
data_prepared = full_pipeline.fit_transform(data)
data_prepared

<44260x177 sparse matrix of type '<class 'numpy.float64'>'
	with 354080 stored elements in Compressed Sparse Row format>

In [14]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation", scores.std())

In [15]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(data_prepared, labels)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [16]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(ridge_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
ridge_reg_rmse_scores = np.sqrt(-scores)

In [17]:
display_scores(ridge_reg_rmse_scores)

Scores: [2082.87681027 1526.01372119 2302.96498768 3331.7783367  1891.83610132
 1919.10364963 1109.5568594  2191.17694533  814.86753377  919.21107886]
Mean: 1808.9386024149298
Standard Deviation 717.7905835064967


In [18]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=0.5)
lasso_reg.fit(data_prepared, labels)

Lasso(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [19]:
scores = cross_val_score(lasso_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
lasso_reg_rmse_scores = np.sqrt(-scores)

In [20]:
display_scores(lasso_reg_rmse_scores)

Scores: [2077.80582305 1508.65993324 2285.4602354  3333.07566411 1901.44322406
 1925.5334023  1120.67019106 2189.58101293  824.09527105  931.32995113]
Mean: 1809.7654708326968
Standard Deviation 713.6794454941377


In [21]:
from sklearn.linear_model import ElasticNet

elastic_reg = ElasticNet(alpha=0.5)
elastic_reg.fit(data_prepared, labels)

ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [22]:
scores = cross_val_score(elastic_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
elastic_reg_rmse_scores = np.sqrt(-scores)

In [23]:
display_scores(elastic_reg_rmse_scores)

Scores: [2572.31767826 2134.28529809 2837.04322475 3515.18429232 2094.86866452
 1981.42176015 1203.70508342 2293.40696851  924.65780401  639.93906835]
Mean: 2019.68298423845
Standard Deviation 841.0574119143745


In [24]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=False)
data_poly = poly_features.fit_transform(data_prepared.toarray())

KeyboardInterrupt: 

In [ ]:
from sklearn.linear_model import 

lin_poly_reg = LinearRegression()
lin_reg.fit(data_poly, labels)